## 1. Creating and Managing Threads

In [1]:
import threading
import time

print("=" * 50)
print("BASIC THREADING")
print("=" * 50)

# Method 1: Using target function
def print_numbers(name, count):
    for i in range(count):
        print(f"[{name}] Number {i}")
        time.sleep(0.5)

t1 = threading.Thread(target=print_numbers, args=("Thread-1", 3))
t2 = threading.Thread(target=print_numbers, args=("Thread-2", 3))

# Start threads
t1.start()
t2.start()

# Wait for completion
t1.join()
t2.join()

print("\nAll threads completed!")

BASIC THREADING
[Thread-1] Number 0
[Thread-2] Number 0
[Thread-1] Number 1[Thread-2] Number 1

[Thread-2] Number 2[Thread-1] Number 2


All threads completed!


## 2. Custom Thread Class

In [2]:
import threading
import time

print("\n" + "=" * 50)
print("CUSTOM THREAD CLASS")
print("=" * 50)

class CustomThread(threading.Thread):
    def __init__(self, name, iterations):
        super().__init__()
        self.name = name
        self.iterations = iterations
    
    def run(self):
        """Override run() method"""
        print(f"[{self.name}] Starting...")
        for i in range(self.iterations):
            print(f"[{self.name}] Iteration {i+1}")
            time.sleep(0.3)
        print(f"[{self.name}] Finished")

# Create and start threads
threads = [
    CustomThread("Worker-1", 2),
    CustomThread("Worker-2", 2)
]

for t in threads:
    t.start()

for t in threads:
    t.join()


CUSTOM THREAD CLASS
[Worker-1] Starting...
[Worker-1] Iteration 1
[Worker-2] Starting...
[Worker-2] Iteration 1
[Worker-1] Iteration 2
[Worker-2] Iteration 2
[Worker-1] Finished
[Worker-2] Finished


## 3. Thread-Safe Data Sharing

In [3]:
import threading
import time

print("\n" + "=" * 50)
print("THREAD-SAFE DATA SHARING")
print("=" * 50)

class BankAccount:
    def __init__(self, balance=0):
        self.balance = balance
        self.lock = threading.Lock()
    
    def deposit(self, amount):
        with self.lock:
            print(f"Depositing {amount}...")
            temp = self.balance
            time.sleep(0.1)  # Simulate processing
            temp += amount
            self.balance = temp
            print(f"New balance: {self.balance}")
    
    def withdraw(self, amount):
        with self.lock:
            if self.balance >= amount:
                print(f"Withdrawing {amount}...")
                temp = self.balance
                time.sleep(0.1)  # Simulate processing
                temp -= amount
                self.balance = temp
                print(f"New balance: {self.balance}")
            else:
                print(f"Insufficient funds!")

# Create account and threads
account = BankAccount(1000)

def customer_operations(account, name):
    for _ in range(2):
        account.deposit(100)
        account.withdraw(50)

threads = [
    threading.Thread(target=customer_operations, args=(account, "Customer-1")),
    threading.Thread(target=customer_operations, args=(account, "Customer-2"))
]

for t in threads:
    t.start()

for t in threads:
    t.join()

print(f"\nFinal balance: {account.balance}")


THREAD-SAFE DATA SHARING
Depositing 100...
New balance: 1100
Depositing 100...
New balance: 1200
Withdrawing 50...
New balance: 1150
Withdrawing 50...
New balance: 1100
Depositing 100...
New balance: 1200
Depositing 100...
New balance: 1300
Withdrawing 50...
New balance: 1250
Withdrawing 50...
New balance: 1200

Final balance: 1200


## 4. Producer-Consumer Pattern

In [4]:
import threading
from queue import Queue
import time
import random

print("\n" + "=" * 50)
print("PRODUCER-CONSUMER PATTERN")
print("=" * 50)

# Queue is thread-safe
data_queue = Queue(maxsize=5)

def producer(name, count):
    for i in range(count):
        item = f"Item-{i}"
        data_queue.put(item)
        print(f"[{name}] Produced: {item}")
        time.sleep(random.random())
    print(f"[{name}] Production complete")

def consumer(name, count):
    consumed = 0
    while consumed < count:
        item = data_queue.get()
        print(f"[{name}] Consumed: {item}")
        time.sleep(random.random())
        consumed += 1
    print(f"[{name}] Consumption complete")

# Create threads
prod = threading.Thread(target=producer, args=("Producer", 5))
cons = threading.Thread(target=consumer, args=("Consumer", 5))

prod.start()
cons.start()

prod.join()
cons.join()

print("\nProducer-Consumer pattern complete!")


PRODUCER-CONSUMER PATTERN
[Producer] Produced: Item-0
[Consumer] Consumed: Item-0
[Producer] Produced: Item-1
[Producer] Produced: Item-2
[Consumer] Consumed: Item-1
[Producer] Produced: Item-3
[Producer] Produced: Item-4
[Consumer] Consumed: Item-2
[Producer] Production complete
[Consumer] Consumed: Item-3
[Consumer] Consumed: Item-4
[Consumer] Consumption complete

Producer-Consumer pattern complete!


## 5. Threading Events

In [5]:
import threading
import time

print("\n" + "=" * 50)
print("THREADING EVENTS")
print("=" * 50)

# Event for synchronization
start_event = threading.Event()

def worker(name):
    print(f"[{name}] Waiting for start signal...")
    start_event.wait()  # Block until event is set
    print(f"[{name}] Event triggered! Starting work...")
    time.sleep(1)
    print(f"[{name}] Work complete")

# Create worker threads
threads = [threading.Thread(target=worker, args=(f"Worker-{i}",)) for i in range(3)]

for t in threads:
    t.start()

time.sleep(2)
print("\n>>> Triggering event...\n")
start_event.set()  # Signal all waiting threads

for t in threads:
    t.join()

print("\nAll workers finished!")


THREADING EVENTS
[Worker-0] Waiting for start signal...
[Worker-1] Waiting for start signal...
[Worker-2] Waiting for start signal...

>>> Triggering event...

[Worker-2] Event triggered! Starting work...
[Worker-1] Event triggered! Starting work...
[Worker-0] Event triggered! Starting work...
[Worker-1] Work complete
[Worker-2] Work complete
[Worker-0] Work complete

All workers finished!


## 6. Thread-Local Storage

In [6]:
import threading
import time

print("\n" + "=" * 50)
print("THREAD-LOCAL STORAGE")
print("=" * 50)

# Thread-local storage
local = threading.local()

def worker(worker_id):
    # Each thread has its own copy
    local.value = worker_id
    local.data = f"Data-{worker_id}"
    
    print(f"Thread {threading.current_thread().name}: value={local.value}, data={local.data}")
    time.sleep(1)
    print(f"Thread {threading.current_thread().name}: value={local.value}, data={local.data}")

threads = [threading.Thread(target=worker, args=(i,)) for i in range(3)]

for t in threads:
    t.start()

for t in threads:
    t.join()

print("\nEach thread maintained its own data!")


THREAD-LOCAL STORAGE
Thread Thread-14 (worker): value=0, data=Data-0
Thread Thread-15 (worker): value=1, data=Data-1
Thread Thread-16 (worker): value=2, data=Data-2
Thread Thread-15 (worker): value=1, data=Data-1
Thread Thread-16 (worker): value=2, data=Data-2
Thread Thread-14 (worker): value=0, data=Data-0

Each thread maintained its own data!


## 7. Daemon Threads

In [ ]:
import threading
import time

print("\n" + "=" * 50)
print("DAEMON THREADS")
print("=" * 50)

def background_task():
    while True:
        print("Background task running...")
        time.sleep(1)

# Create daemon thread
daemon = threading.Thread(target=background_task, daemon=True)
daemon.start()

# Main program
print("Main program running...")
time.sleep(3)
print("Main program ending... Daemon will be killed")

# Program exits, daemon thread is killed automatically


DAEMON THREADS
Background task running...
Main program running...
Background task running...
Background task running...
Main program ending... Daemon will be killed


Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
Background task running...
B

## 8. Thread Conditions

In [8]:
import threading
import time

print("\n" + "=" * 50)
print("CONDITION VARIABLES")
print("=" * 50)

condition = threading.Condition()
data = []

def producer_cond():
    for i in range(3):
        with condition:
            data.append(i)
            print(f"Produced: {i}")
            condition.notify_all()  # Notify waiting threads
            time.sleep(0.5)

def consumer_cond():
    while True:
        with condition:
            while not data:  # While no data
                condition.wait()  # Wait for notification
            
            if data:
                item = data.pop(0)
                print(f"Consumed: {item}")
                if item == 2:
                    break

prod = threading.Thread(target=producer_cond)
cons = threading.Thread(target=consumer_cond)

prod.start()
cons.start()

prod.join()
cons.join()


CONDITION VARIABLES
Produced: 0
Produced: 1
Produced: 2
Consumed: 0
Consumed: 1
Consumed: 2


## 9. Performance Comparison

In [9]:
import threading
import time

print("\n" + "=" * 50)
print("THREADING PERFORMANCE")
print("=" * 50)

# I/O-bound operation
def io_operation(duration):
    time.sleep(duration)

# Sequential
start = time.time()
for i in range(5):
    io_operation(1)
seq_time = time.time() - start
print(f"Sequential: {seq_time:.2f}s")

# Threaded
start = time.time()
threads = [threading.Thread(target=io_operation, args=(1,)) for i in range(5)]
for t in threads:
    t.start()
for t in threads:
    t.join()
thread_time = time.time() - start
print(f"Threaded: {thread_time:.2f}s")
print(f"Speedup: {seq_time / thread_time:.1f}x")


THREADING PERFORMANCE
Sequential: 5.00s
Threaded: 1.00s
Speedup: 5.0x
